In [1]:
from levelwise_model.levelwise_model import LevelwiseModel
from levelwise_model.config import Config, Word2VecConfig, SentencePieceConfig
from levelwise_model.test_bench import LSTestBench

In [ ]:
tag = "original_20k_150x1__60k_250x5__50k_250x5"

In [3]:
lw_model = LevelwiseModel(
    tag=tag,
)

In [4]:
configs = [
    Config(
        SentencePieceConfig(max_sentence_length=20000, vocab_size=20000, model_type="unigram"),
        Word2VecConfig(vector_size=150, window=1),
        cluster_threshold=0.45
    ),
    Config(
        SentencePieceConfig(max_sentence_length=5000, vocab_size=60000, model_type="unigram"),
        Word2VecConfig(vector_size=250, window=5),
        cluster_threshold=0.9
    ),
    Config(
        SentencePieceConfig(max_sentence_length=5000, vocab_size=50000, model_type="unigram"),
        Word2VecConfig(vector_size=250, window=5),
        cluster_threshold=1
    )
]

In [5]:
test_bench = LSTestBench(scores_file="data/level_wise/level0/pairs.txt")

In [6]:
lw_model.train(
    input_file="data/level_wise/level0/corpus_original.txt",
    utterance_file="data/level_wise/level0/utterances_original.txt",
    n_levels=2,
    configs=configs,
    test_bench=test_bench
)